In [116]:
import tensorflow.keras as tf 
import tensorflow as tf1

Lenet is the first Neural Network model to be implemented for the Computer Vision task, but it is useful for the grayscale images. It is used for the bank system to identify the handwritten digits. 

In [117]:
# Architecture of LeNet

In [118]:
lenet_model = tf.Sequential()

In [119]:
lenet_model.add(
    tf.layers.Input(shape = (28, 28, 1))
)

In [120]:
lenet_model.add(
    tf.layers.Conv2D(
        kernel_size = (5, 5), strides = (1, 1), activation = 'relu', filters = 6
    )
)

In [121]:
lenet_model.add(
    tf.layers.AveragePooling2D(pool_size = (2, 2), strides = (2, 2))
)

In [122]:
lenet_model.add(
    tf.layers.Conv2D(
        kernel_size = (5, 5), strides = (1, 1), activation = 'relu', filters = 16
    )
)

In [123]:
lenet_model.add(
    tf.layers.AveragePooling2D(pool_size = (2, 2), strides = (2, 2))
)

In [124]:
lenet_model.add(
    tf.layers.Flatten()
)

In [125]:
lenet_model.add(
    tf.layers.Dense(units = 120, activation = 'relu')
)

In [126]:
lenet_model.add(
    tf.layers.Dense(units = 84, activation = 'relu')
)

In [127]:
lenet_model.add(
    tf.layers.Dense(units = 10,)
)

In [128]:
lenet_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 24, 24, 6)         156       
                                                                 
 average_pooling2d_8 (Averag  (None, 12, 12, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_16 (Conv2D)          (None, 8, 8, 16)          2416      
                                                                 
 average_pooling2d_9 (Averag  (None, 4, 4, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten_6 (Flatten)         (None, 256)               0         
                                                                 
 dense_14 (Dense)            (None, 120)              

In [129]:
# Prepare the train and test dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = tf.datasets.mnist.load_data()

# Normalize data
x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))


In [130]:
lenet_model.compile(loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True), metrics=[tf.metrics.SparseCategoricalAccuracy()], optimizer = 'adam')

In [131]:
history = lenet_model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 5)

Epoch 1/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.2380 - sparse_categorical_accuracy: 0.9265 - val_loss: 0.0860 - val_sparse_categorical_accuracy: 0.9716
Epoch 2/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0785 - sparse_categorical_accuracy: 0.9759 - val_loss: 0.0547 - val_sparse_categorical_accuracy: 0.9832
Epoch 3/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0561 - sparse_categorical_accuracy: 0.9825 - val_loss: 0.0435 - val_sparse_categorical_accuracy: 0.9854
Epoch 4/5
1875/1875 [==============================] - 29s 16ms/step - loss: 0.0453 - sparse_categorical_accuracy: 0.9855 - val_loss: 0.0392 - val_sparse_categorical_accuracy: 0.9873
Epoch 5/5
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0360 - sparse_categorical_accuracy: 0.9886 - val_loss: 0.0545 - val_sparse_categorical_accuracy: 0.9841


We are going to use the model of lenet as a teacher and we are going to create a student model with the knowledge of the teacher 

In [133]:
lenet_model.save('lenet.h5')


In [134]:
import os 

(os.stat('/content/lenet.h5').st_size / 1024)/1024

0.5589599609375

Not a Quite big model, but this is enough for the demonstration


In [140]:
student = tf.Sequential(
    [
        tf.layers.Input(shape=(28, 28, 1)),
        tf.layers.Conv2D(16, (3, 3), strides=(2, 2), padding="same"),
        tf.layers.LeakyReLU(alpha=0.2),
        tf.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"),
        tf.layers.Conv2D(32, (3, 3), strides=(2, 2), padding="same"),
        tf.layers.Flatten(),
        tf.layers.Dense(10),
    ],
    name="student",
)


In [141]:
# Creating a copy of the model

student_copy = tf.models.clone_model(student)

In [142]:
# model summary of the student


student.summary()

Model: "student"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 14, 14, 16)        160       
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 16)       0         
 2D)                                                             
                                                                 
 conv2d_20 (Conv2D)          (None, 7, 7, 32)          4640      
                                                                 
 flatten_8 (Flatten)         (None, 1568)              0         
                                                                 
 dense_18 (Dense)            (None, 10)                15690     
                                                           

Parameter Wise, Teacher Model has a 60k parameters, Student has a 20k parameters. So there is some difference in the number of parameter between the teacher and the student. 
We are going to use the distiller cklass which is defined in keras. https://keras.io/examples/vision/knowledge_distillation/ . This is the link for the example of knowledge distillation, take a look at it. This is created based on the example from the keras offical page

In [145]:
class Distiller(tf.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf1.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf1.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf1.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [146]:
distiller = Distiller(student=student, teacher=lenet_model)
distiller.compile(
    optimizer=tf.optimizers.Adam(),
    metrics=[tf.metrics.SparseCategoricalAccuracy()],
    student_loss_fn=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=tf.losses.KLDivergence(),
    alpha=0.1,
    temperature=10,
)

# Distill teacher to student
distiller.fit(x_train, y_train, epochs=3)

# Evaluate student on test dataset
distiller.evaluate(x_test, y_test)

Epoch 1/3
1875/1875 [==============================] - 43s 22ms/step - sparse_categorical_accuracy: 0.9229 - student_loss: 0.2544 - distillation_loss: 0.0491
Epoch 2/3
1875/1875 [==============================] - 27s 14ms/step - sparse_categorical_accuracy: 0.9686 - student_loss: 0.1009 - distillation_loss: 0.0241
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - sparse_categorical_accuracy: 0.9756 - student_loss: 0.0726


[0.975600004196167, 0.005354690365493298]

In [148]:
# Train student as doen usually
student_copy.compile(
    optimizer=tf.optimizers.Adam(),
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.metrics.SparseCategoricalAccuracy()],
)

# Train and evaluate student trained from scratch.
student_copy.fit(x_train, y_train, epochs=3)
student_copy.evaluate(x_test, y_test)


Epoch 1/3
1875/1875 [==============================] - 19s 10ms/step - loss: 0.2425 - sparse_categorical_accuracy: 0.9282
Epoch 2/3
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0924 - sparse_categorical_accuracy: 0.9717
Epoch 3/3
313/313 [==============================] - 1s 4ms/step - loss: 0.0711 - sparse_categorical_accuracy: 0.9771


[0.07109558582305908, 0.9771000146865845]

In [149]:
# Both the student from scratch model and distilled model provides the similar result. But this is not the case in the complex dataset like a image 
# dataset, classication, object detection, in those tasks, knowledge distillation performs well

In [154]:
(os.path.getsize(filename='/content/lenet.h5')/(1024 * 1024))

0.5589599609375

In [155]:
# Converting tf model into tf-lite model 
# step1 : Create a convertor
# Extension of tf-lite model is .tflite

tflite_filename = "tflite-model.tflite"

# This will convert our model into a tflite model. We defined the class object, inorder to convert, we need to call the method .convert
tflite_convertor = tf1.lite.TFLiteConverter.from_keras_model(lenet_model)
tflite_model = tflite_convertor.convert()


INFO:tensorflow:Assets written to: /tmp/tmpbog7c9a1/assets


In [156]:
open(tflite_filename, "wb").write(tflite_model)

180988

In [157]:
(os.path.getsize(filename='/content/tflite-model.tflite')/(1024 * 1024))

0.17260360717773438